In [1]:
import random
import numpy as np
import torch
import scipy.sparse as sp
import os
from utils import csr2torch, recall_at_k, ndcg_at_k, normalize_sparse_adjacency_matrix

random.seed(2022)
np.random.seed(2022)

In [5]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda")
current_directory = os.getcwd()

In [10]:
verbose = 1  # 0: no print, 1: print results
dataset = "gowalla"  # Either gowalla, yelp, amazon, or ml-1m
alpha = 0.6
power = 0.7
filter = 1 # 1: linear 2: 2nd-order 3: 3rd-order ideal LPF


if verbose:
    print(f"Device: {device}")
dataset = dataset
path_tr = f"{current_directory}/dataset/{dataset}_train.npz"
path_ts = f"{current_directory}/dataset/{dataset}_test.npz"
R_tr = csr2torch(sp.load_npz(path_tr)).to(device)
R_ts = csr2torch(sp.load_npz(path_ts)).to(device)

n_users = R_tr.shape[0]
n_items = R_tr.shape[1]
if verbose:
    print(f"number of users: {n_users}")
    print(f"number of items: {n_items}")

n_inters = torch.nonzero(R_tr._values()).cpu().size(0) + torch.nonzero(
    R_ts[0]._values()
).cpu().size(0)

if verbose:
    print(f"number of overall ratings: {n_inters}")

R_norm = normalize_sparse_adjacency_matrix(R_tr.to_dense(), alpha)
R = R_tr.to_dense()
P = R_norm.T @ R_norm
P.data **= power
P = P.to(device=device).float()
R = R.to(device=device).float()

if filter == 1:
    results = R @ (P)
elif filter == 2:
    results = R @ (2*P-P@P)
elif filter == 3:
    results = R @ (P + 0.01*(-P@P@P +10*P@P - 29*P))
# Now get the results
gt_mat = R_ts.to_dense()
results = results + (-99999) * R_tr.to_dense()
gt_mat = gt_mat.cpu().detach().numpy()
results = results.cpu().detach().numpy()

print(f"Recall@20: {recall_at_k(gt_mat, results, k=20):.4f} \n")
print(f"NDCG@20: {ndcg_at_k(gt_mat, results, k=20):.4f} \n")

Device: cpu
number of users: 29858
number of items: 40981
number of overall ratings: 810160


KeyboardInterrupt: 